In [1]:
%pip install discopula


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


> Make sure to have discopula's latest version installed using `pip`. More information about the latest version can be found at https://pypi.org/project/discopula/

Run the following upgrade commands on your terminal if you are facing issues related to `pip` or `scipy`

```
# pip install --upgrade pip
# pip install --upgrade scipy
```

In [2]:
import numpy as np
from discopula import GenericCheckerboardCopula

# 2-Dimensional Case 

### Create Sample Contingency Table and Initialize the GenericCheckerboardCopula

The `GenericCheckerboardCopula` object can be initialized with a contingency table represented as a NumPy array. For a 2D contingency table:

- `axis=0`: First variable ($X_1$) with 5 categories (e.g. as per the 2D example in JMA 2021 paper)
- `axis=1`: Second variable ($X_2$) with 3 categories (e.g. as per the 2D example in JMA 2021 paper)

The axis indexing follows NumPy's convention, starting from the outermost dimension (0). As described in Joe, Liu & Manuguerra (2021), the variables are ordered such that:

- $X_1$ corresponds to `axis=0` (rows)
- $X_2$ corresponds to `axis=1` (columns)

This ordering is important for calculating directional measures of association between variables. And, later on in this `.ipynb`, you can see how we can conveniently mention the 1-indexed variable numbers to perform association measure calculation, regression, and prediction.

In [3]:

contingency_table = np.array([
    [0, 0, 20],
    [0, 10, 0],
    [20, 0, 0],
    [0, 10, 0],
    [0, 0, 20]
])
copula = GenericCheckerboardCopula.from_contingency_table(contingency_table)
print(f"Shape of the inferred joint probability matrix P: {copula.P.shape}")
print(f"Probability matrix P:\n{copula.P}")

Shape of the inferred joint probability matrix P: (5, 3)
Probability matrix P:
[[0.    0.    0.25 ]
 [0.    0.125 0.   ]
 [0.25  0.    0.   ]
 [0.    0.125 0.   ]
 [0.    0.    0.25 ]]


### Calculating CCRAM & SCCRAM (non-vectorized)

In [4]:
ccram_X1_to_X2 = copula.calculate_CCRAM(predictors=[1], response=2)
ccram_X2_to_X1 = copula.calculate_CCRAM(predictors=[2], response=1)
print(f"CCRAM X1 to X2: {ccram_X1_to_X2:.4f}")
print(f"CCRAM X2 to X1: {ccram_X2_to_X1:.4f}")

sccram_X1_to_X2 = copula.calculate_CCRAM(predictors=[1], response=2, scaled=True)
sccram_X2_to_X1 = copula.calculate_CCRAM(predictors=[2], response=1, scaled=True)
print(f"SCCRAM X1 to X2: {sccram_X1_to_X2:.4f}")
print(f"SCCRAM X2 to X1: {sccram_X2_to_X1:.4f}")

CCRAM X1 to X2: 0.8438
CCRAM X2 to X1: 0.0000
SCCRAM X1 to X2: 1.0000
SCCRAM X2 to X1: 0.0000


### Getting Category Predictions

In [5]:
predictions_X1_to_X2 = copula.get_category_predictions_multi(predictors=[1], response=2)
print("\nPredictions from X1 to X2:")
print(predictions_X1_to_X2)

axis_to_name_dict = {1: "Income Bracket", 2: "Education Level"}
predictions_Education_to_Income = copula.get_category_predictions_multi(predictors=[2], response=1, axis_names=axis_to_name_dict)
print("\nPredictions from Education Level to Income Bracket:")
print(predictions_Education_to_Income)


Predictions from X1 to X2:
   X1 Category  Predicted X2 Category
0            1                      3
1            2                      2
2            3                      1
3            4                      2
4            5                      3

Predictions from Education Level to Income Bracket:
   Education Level Category  Predicted Income Bracket Category
0                         1                                  3
1                         2                                  3
2                         3                                  3


### Calculating Scores and their Variances

In [7]:
# Calculate and display scores for both axes
scores_X1 = copula.calculate_scores(1)
scores_X2 = copula.calculate_scores(2)

print("Scores for X1:")
print(scores_X1)
# Expected: [0.125, 0.3125, 0.5, 0.6875, 0.875]

print("\nScores for X2:")
print(scores_X2)
# Expected: [0.125, 0.375, 0.75]

# Calculate and display variance of scores
variance_S_X1 = copula.calculate_variance_S(0)
variance_S_X2 = copula.calculate_variance_S(1)

print("\nVariance of scores for X1:", variance_S_X1)
# Expected: 81/1024 = 0.0791015625
print("Variance of scores for X2:", variance_S_X2)
# Expected: 9/128 = 0.0703125 

Scores for X1:
[np.float64(0.125), np.float64(0.3125), np.float64(0.5), np.float64(0.6875), np.float64(0.875)]

Scores for X2:
[np.float64(0.125), np.float64(0.375), np.float64(0.75)]

Variance of scores for X1: 0.0791015625
Variance of scores for X2: 0.0703125


# 4-Dimensional Case (Real Data Analysis from JMA2021)

### Create Sample Data in Cases Form and Initialize the GenericCheckerboardCopula

The `GenericCheckerboardCopula` can be initialized using categorical data with multiple variables. Let's explain this with a concrete example:

Consider a dataset with 4 categorical variables:
- Length of Previous Attack ($X_1$): 2 categories (Short, Long)
- Pain Change ($X_2$): 3 categories (Worse, Same, Better)
- Lordosis ($X_3$): 2 categories (absent/decreasing, present/increasing)
- Back Pain ($X_4$): 6 categories (Worse (W), Same (S), Slight Improvement (SI), Moderate Improvement (MODI), Marked Improvement (MARI), Complete Relief (CR))

In the data structure:
- Each row represents one observation
- Each column represents one categorical variable
- The data is stored as a NumPy array of categorical values starting from 0 to (number of categories - 1)

When creating the copula:
- Variables are numbered from $X_1$ to $X_4$
- Internal indexing starts from 0 (Python convention):
  * $X_1$ → `axis=0` (2 categories)
  * $X_2$ → `axis=1` (3 categories)
  * $X_3$ → `axis=2` (2 categories)
  * $X_4$ → `axis=3` (6 categories)

In [8]:
real_cases_data = np.array([
    # RDA Row 1
    [0,2,0,1],[0,2,0,4],[0,2,0,4],
    [0,2,0,5], [0,2,0,5],[0,2,0,5],[0,2,0,5],
    # RDA Row 2
    [0,2,1,3],[0,2,1,4],[0,2,1,4],[0,2,1,4],
    # RDA Row 3
    [0,1,0,1],[0,1,0,1],[0,1,0,2],[0,1,0,2],[0,1,0,2],
    [0,1,0,4],[0,1,0,4],[0,1,0,4],[0,1,0,4],[0,1,0,4],[0,1,0,4],
    [0,1,0,5],[0,1,0,5],[0,1,0,5],[0,1,0,5],
    # RDA Row 4
    [0,1,1,1],[0,1,1,3],[0,1,1,3],[0,1,1,5],
    # RDA Row 5
    [0,0,0,4],[0,0,0,4],[0,0,0,5],[0,0,0,5],
    # RDA Row 6
    [0,0,1,2],[0,0,1,3],[0,0,1,4],[0,0,1,4],[0,0,1,4],
    # RDA Row 7
    [1,2,0,2],[1,2,0,2],[1,2,0,2],[1,2,0,4],[1,2,0,5],[1,2,0,5],
    # RDA Row 8
    [1,2,1,1],[1,2,1,4],[1,2,1,4],[1,2,1,4],
    # RDA Row 9
    [1,1,0,1],[1,1,0,1],[1,1,0,1],[1,1,0,2],[1,1,0,2],[1,1,0,2],[1,1,0,2],
    [1,1,0,3],[1,1,0,3],[1,1,0,3],[1,1,0,3],[1,1,0,3],
    [1,1,0,4],[1,1,0,4],[1,1,0,4],[1,1,0,4],[1,1,0,4],[1,1,0,4],
    [1,1,0,5],[1,1,0,5],
    # RDA Row 10
    [1,1,1,0],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],
    [1,1,1,2],[1,1,1,2],[1,1,1,2],[1,1,1,2],
    [1,1,1,3],[1,1,1,3],[1,1,1,3],[1,1,1,5],
    # RDA Row 11
    [1,0,0,0],[1,0,0,0],[1,0,0,1],[1,0,0,1],[1,0,0,2],
    [1,0,0,3],[1,0,0,3],[1,0,0,3],[1,0,0,3],[1,0,0,3],
    [1,0,0,4],[1,0,0,4],
    # RDA Row 12
    [1,0,1,0],[1,0,1,0],[1,0,1,2],[1,0,1,2],
    [1,0,1,3],[1,0,1,3],[1,0,1,3]
])
rda_copula = GenericCheckerboardCopula.from_cases(cases=real_cases_data, shape=(2,3,2,6))
print(f"Shape of the inferred joint probability matrix P: {rda_copula.P.shape}")
print(f"Probability matrix P:\n{rda_copula.P}\n")
print(f"Marginal pdfs:\n{rda_copula.marginal_pdfs}\n")
print(f"Marginal cdfs:\n{rda_copula.marginal_cdfs}")

Shape of the inferred joint probability matrix P: (2, 3, 2, 6)
Probability matrix P:
[[[[0.         0.         0.         0.         0.01980198 0.01980198]
   [0.         0.         0.00990099 0.00990099 0.02970297 0.        ]]

  [[0.         0.01980198 0.02970297 0.         0.05940594 0.03960396]
   [0.         0.00990099 0.         0.01980198 0.         0.00990099]]

  [[0.         0.00990099 0.         0.         0.01980198 0.03960396]
   [0.         0.         0.         0.00990099 0.02970297 0.        ]]]


 [[[0.01980198 0.01980198 0.00990099 0.04950495 0.01980198 0.        ]
   [0.01980198 0.         0.01980198 0.02970297 0.         0.        ]]

  [[0.         0.02970297 0.03960396 0.04950495 0.05940594 0.01980198]
   [0.00990099 0.03960396 0.03960396 0.02970297 0.         0.00990099]]

  [[0.         0.         0.02970297 0.         0.00990099 0.01980198]
   [0.         0.00990099 0.         0.         0.02970297 0.        ]]]]

Marginal pdfs:
{0: array([0.38613861, 0.6138613

### Calculating CCRAM & SCCRAM (non-vectorized)

In [9]:
rda_ccram_X1_X2_X3_to_X4 = rda_copula.calculate_CCRAM(predictors=[1, 2, 3], response=4)
print(f"CCRAM (X1, X2, X3) -> X4: {rda_ccram_X1_X2_X3_to_X4:.4f}")

rda_sccram_X1_X2_X3_to_X4 = rda_copula.calculate_CCRAM(predictors=[1, 2, 3], response=4, scaled=True)
print(f"SCCRAM (X1, X2, X3) -> X4: {rda_sccram_X1_X2_X3_to_X4:.4f}")

CCRAM (X1, X2, X3) -> X4: 0.2604
SCCRAM (X1, X2, X3) -> X4: 0.2716


### Getting Category Predictions

In [10]:
rda_predictions_X1_X2_X3_to_X4 = rda_copula.get_category_predictions_multi(predictors=[1, 2, 3], response=4)
print("\nPredictions (X1, X2, X3) -> X4:")
print(rda_predictions_X1_X2_X3_to_X4)

rda_axis_to_name_dict = {1: "Length of Previous Attack", 2: "Pain Change", 3: "Lordosis", 4: "Back Pain"}
rda_predictions_X1_X2_X3_to_X4_named = rda_copula.get_category_predictions_multi(predictors=[1, 2, 3], response=4, axis_names=rda_axis_to_name_dict)
print("\nPredictions (X1, X2, X3) -> X4 with custom names:")
print(rda_predictions_X1_X2_X3_to_X4_named)


Predictions (X1, X2, X3) -> X4:
    X1 Category  X2 Category  X3 Category  Predicted X4 Category
0             1            1            1                      5
1             1            1            2                      5
2             1            2            1                      5
3             1            2            2                      4
4             1            3            1                      5
5             1            3            2                      5
6             2            1            1                      3
7             2            1            2                      3
8             2            2            1                      4
9             2            2            2                      3
10            2            3            1                      4
11            2            3            2                      4

Predictions (X1, X2, X3) -> X4 with custom names:
    Length of Previous Attack Category  Pain Change Category  \
0      

### Calculating Scores and their Variances

In [ ]:
# Calculate and display scores for both axes
rda_scores_X1 = rda_copula.calculate_scores(1)
rda_scores_X2 = rda_copula.calculate_scores(2)
rda_scores_X3 = rda_copula.calculate_scores(3)
rda_scores_X4 = rda_copula.calculate_scores(4)

print("Scores for X1:")
print(rda_scores_X1)
print("\nScores for X2:")
print(rda_scores_X2)
print("\nScores for X3:")
print(rda_scores_X3)
print("\nScores for X4:")
print(rda_scores_X4)

# Calculate and display variance of scores
rda_variance_S_X1 = rda_copula.calculate_variance_S(1)
rda_variance_S_X2 = rda_copula.calculate_variance_S(2)
rda_variance_S_X3 = rda_copula.calculate_variance_S(3)
rda_variance_S_X4 = rda_copula.calculate_variance_S(4)

print("\nVariance of scores for axis 0:", rda_variance_S_X1)
print("\nVariance of scores for axis 1:", rda_variance_S_X2)
print("\nVariance of scores for axis 2:", rda_variance_S_X3)
print("\nVariance of scores for axis 3:", rda_variance_S_X4)
# Expected 12 * (variance of scores for axis 3): 0.07987568681385342*12 = 0.95850824176

Scores for X1:
[np.float64(0.19306930693069307), np.float64(0.693069306930693)]

Scores for X2:
[np.float64(0.13861386138613863), np.float64(0.5346534653465347), np.float64(0.8960396039603961)]

Scores for X3:
[np.float64(0.3168316831683168), np.float64(0.8168316831683167)]

Scores for X4:
[np.float64(0.024752475247524754), np.float64(0.1188118811881188), np.float64(0.27722772277227725), np.float64(0.4653465346534653), np.float64(0.7029702970297029), np.float64(0.9207920792079207)]


KeyError: 4